In [8]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
classifier("This is a guide about accounting for inventory in business operations.", candidate_labels=["accounting", "inventory management", "finance"])

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
classifier("The process of billing customers for services rendered.", candidate_labels=["billing", "accounts receivable", "finance"])

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
classifier("Accounts payable and accounts receivable are essential components of financial operations.", candidate_labels=["accounts payable", "accounts receivable", "finance"])

generator = pipeline("text-generation", model="openai-gpt")
generator("In this accounting module, we will cover the invoicing process and best practices for")

generator = pipeline("text-generation", model="distilgpt2")
generator("In this accounting tutorial, we will learn about managing accounts payable and receivable, focusing on", max_length=30, num_return_sequences=2, truncation=True)

generator = pipeline("text-generation", model="distilgpt2")
generator("In this course, we will explore inventory accounting and cost management methods")

unmasker = pipeline("fill-mask", model="bert-base-uncased")
unmasker("Inventory accounting includes methods such as [MASK] inventory, weighted average, and FIFO.", top_k=2)

ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", grouped_entities=True)
ner("My name is Sarah and I work at Hugging Face in Brooklyn.")

qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
context = "Invoicing is a critical part of the billing process in accounting, where a bill is issued to a customer for services or goods."
question = "What is invoicing?"

result = qa_pipeline(question=question, context=context)
print(f"Answer: {result['answer']}")

qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
context = "Accounts payable refers to money owed by a business to its suppliers, while accounts receivable represents the amount owed to the business by its customers."
question = "What is accounts payable?"

result = qa_pipeline(question=question, context=context)
print(f"Answer: {result['answer']}")

classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
classifier("The invoicing software is very user-friendly and efficient for tracking payments.")

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summarizer(
    """
    Inventory accounting is a complex process that involves tracking the value of goods
    held by a business. Various methods are used, including FIFO, LIFO, and weighted
    average, each with its own impact on financial statements and tax obligations.
    Inventory accounting affects both the cost of goods sold and the net income reported
    by a company. Proper management of inventory accounting helps businesses make
    informed financial decisions, optimize stock levels, and maintain accurate financial records.
    """,
    max_length=60,
    min_length=30,
    do_sample=False
)

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-id-en")

text_to_translate = "Piutang usaha adalah jumlah yang harus dibayar oleh pelanggan kepada bisnis untuk barang atau layanan yang diberikan."
result = translator(text_to_translate)

print(result[0]['translation_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.

Answer: billing process
Answer: money owed by a business to its suppliers
Business debt is the amount that customers pay to business for the goods or services given.
